In [29]:
import geemap, ee
import seaborn as sns
import matplotlib.pyplot as plt
ee.Initialize()

#Imports annual composites for Europe & Asia
vcm_orm_ntl_2015_asia = ee.Image.loadGeoTIFF("gs://2015_composite_asia/SVDNB_npp_20150101-20151231_75N060E_vcm-orm-ntl_v10_c201701311200.avg_rade9-COG.tif")
vcm_orm_ntl_2016_asia = ee.Image.loadGeoTIFF("gs://2016_composite_asia/SVDNB_npp_20160101-20161231_75N060E_vcm-orm-ntl_v10_c201807311200.avg_rade9-COG.tif")
vcm_orm_ntl_2015_europe = ee.Image.loadGeoTIFF("gs://2015_composite_europe/SVDNB_npp_20150101-20151231_75N060W_vcm-orm-ntl_v10_c201701311200.avg_rade9-COG.tif")
vcm_orm_ntl_2016_europe = ee.Image.loadGeoTIFF("gs://2016_composite_europe/SVDNB_npp_20160101-20161231_75N060W_vcm-orm-ntl_v10_c201807311200.avg_rade9-COG.tif")

continent_collection_2015 = ee.ImageCollection([vcm_orm_ntl_2015_asia, vcm_orm_ntl_2015_europe])
continent_collection_2016 = ee.ImageCollection([vcm_orm_ntl_2016_asia, vcm_orm_ntl_2016_europe])

mosaic_2015 = continent_collection_2015.mosaic()
mosaic_2016 = continent_collection_2016.mosaic()

eog_mask = mosaic_2015.add(mosaic_2016)
eog_mask = eog_mask.mask(eog_mask)

vis_params = {
    'min':0,
    'max':60,
    'palette': ['380245', '23678b', '21908c', '39b185', 'fde725'],
    'opacity': 0.75
}

turkey = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Turkey'))
japan = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Japan'))
uk = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'U.K. of Great Britain and Northern Ireland'))
russia = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Russian Federation'))
mexico = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Mexico')) #NOT IN EOG MASK
bulgaria = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Bulgaria'))

In [30]:
regionBeingAnalyzed = bulgaria

def print_mean_SOL(geemap_image):
    mean_SOL = geemap_image.reduceRegion(
                    reducer = ee.Reducer.mean(), 
                    geometry = regionBeingAnalyzed.geometry(),
                    scale = 500,
                    maxPixels = 1e11)
    for k, v in mean_SOL.getInfo().items():
        print(v)
        
def mask_quality(image):
    quality_mask = image.select('Mandatory_Quality_Flag').lte(0)
    #cloud_mask = image.select("QF_Cloud_Mask").lte(1000)
    #combined_mask = quality_mask.And(cloud_mask)
    return image.updateMask(quality_mask)

dates = [
#                 "2012-05-15", "2012-08-15","2012-11-15",
#    "2013-02-15","2013-05-15", "2013-08-15","2013-11-15",
    "2014-02-15","2014-05-15", "2014-08-15","2014-11-15",
    "2015-02-15","2015-05-15", "2015-08-15","2015-11-15",
    "2016-02-15","2016-05-15", "2016-08-15","2016-11-15",
    "2017-02-15","2017-05-15", "2017-08-15","2017-11-15",
    "2018-02-15","2018-05-15", "2018-08-15","2018-11-15",
    "2019-02-15","2019-05-15", "2019-08-15","2019-11-15",
    "2020-02-15","2020-05-15", "2020-08-15"
]

In [31]:
#                        .map(mask_quality) \
#                        .mask(eog_mask)
for i in range (len(dates)-1):
    viirs = ee.ImageCollection("NOAA/VIIRS/001/VNP46A2") \
                        .filterDate(dates[i], dates[i+1]) \
                        .select('DNB_BRDF_Corrected_NTL') \
                        .median()
    print_mean_SOL(viirs)
print('done')

0.7803949018515801
0.7309423903016745
0.7652237725457317
1.0070248499495746
0.7441530969498965
0.7437938839729927
0.747802651861295
0.7626885096666323
0.7123542904457996
0.7650444499958857
0.7074139340093014
0.958498313626562
0.7800618410808882
0.7362930325540035
0.7509500852931029
0.8289018015540773
0.7551300840641756
0.7479159690899001
0.685283100030367
0.8463036288647228
0.7200070170767042
0.6683217755862845
0.7397838859003165
0.7600115525338207
0.5546695484839939
0.556701951846144
done


In [13]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/001/VNP46A2") \
                        .filterDate("2014-02-15", "2014-05-15") \
                        .select('DNB_BRDF_Corrected_NTL') \
                        .median() \
                        .mask(eog_mask)
print('check')
print_mean_SOL(viirs_quarterly_clean)

check
3.166772068147594


In [7]:
viirs_quarterly_NOMASK = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                        .filterDate("2014-05-15", "2014-08-15") \
                        .select('avg_rad') \
                        .median() \
                        .mask(eog_mask)
    #print('Average between ' + dates[i] + ' & ' + dates[i+1] + ':')
print_mean_SOL(viirs_quarterly_NOMASK)

2.930363611429936


In [3]:
for i in range (0, len(dates)-1):
    viirs_quarterly_NOMASK = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                        .filterDate(dates[i], dates[i+1]) \
                        .select('avg_rad') \
                        .median() \
                        #.mask(eog_mask)
    #print('Average between ' + dates[i] + ' & ' + dates[i+1] + ':')
    print_mean_SOL(viirs_quarterly_NOMASK)
print('done')

0.8309202224651786
0.8747715846122801
0.8194935946067913
1.2052244797481517
0.8736545339384783
0.8021302474291561
0.9128671616145454
1.0208848280228338
0.908879174878955
0.8848844089516386
1.008441238105163
1.4216621776964002
1.2144171876539724
1.1843728904128987
1.3241486869041819
1.2779496320561066
1.2985802883323603
1.2768222861274656
1.299089803026718
1.4907542011227195
1.375474888363225
1.3084524082211046
1.442269723380267
1.5109479235240497
1.4285631939566141
1.4818822738233641
done


In [14]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

{'avg_rad': 5.423455522793967}


In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [5]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)

In [ ]:
viirs_quarterly_clean = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                    .filterDate("2019-12-01", "2019-12-31") \
                    .select('avg_rad').mean().mask(eog_mask)

print_mean_SOL(viirs_quarterly_clean)